# Using AI21 Grammatical Error Correction (GEC) on SageMaker through Model Packages

This sample notebook shows you how to deploy **AI21 Grammatical Error Correction (GEC)** using Amazon SageMaker.


--------------------
## <font color='orange'>Important:</font>
Please visit model detail page in <a href="https://aws.amazon.com/marketplace/pp/prodview-dplsurctqbgf6 ">https://aws.amazon.com/marketplace/pp/prodview-dplsurctqbgf6</a> to learn more. <font color='orange'>If you do not have access to the link, please contact account admin for the help.</font>

You will find details about the model including pricing, supported region, and end user license agreement. To use the model, please click “<font color='orange'>Continue to Subscribe</font>” from the detail page, come back here and learn how to deploy and inference.


-------------------


## Pre-requisites:
1. Before running this notebook, please make sure you got this notebook from the model catalog on SageMaker AWS Management Console.
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**.
1. This notebook is intended to work with **boto3 v1.25.4** or higher.

## Contents:
1. [Select model package](#1.-Select-model-package)
1. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   1. [Proofread and detect mistakes](#B.-Proofread-and-detect-mistakes)
1. [Clean-up](#3.-Clean-up)
   1. [Delete the endpoint](#A.-Delete-the-endpoint)
   1. [Delete the model](#B.-Delete-the-model)


## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## Imports

In [ ]:
import json
from sagemaker import ModelPackage
from sagemaker import get_execution_role
import sagemaker as sage
import boto3

### Check the version of boto3 - must be v1.25.4 or higher
If you see a lower version number, pick another kernel to run the notebook, with Python 3.8 or above

In [ ]:
boto3.__version__

### Install ai21 python SDK

In [ ]:
! pip install -U "ai21[AWS]>=2.1.2"
import ai21

### Check the version of ai21 - must be 2.1.2 or higher

In [ ]:
import ai21
ai21.__version__

## 1. Select model package
Confirm that you received this notebook from the model catalog in SageMaker AWS Management Console.

In [ ]:
region = boto3.Session().region_name

# Get the updated ARN
model_package_arn = ai21.SageMaker.get_model_package_arn(model_name="gec", region=region)

In [ ]:
role = get_execution_role()
sagemaker_session = sage.Session()

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html).

In [ ]:
endpoint_name = "gec"

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.g4dn.2xlarge"
)

### A. Create an endpoint

In [ ]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type,
    endpoint_name=endpoint_name, 
    model_data_download_timeout=3600,
    container_startup_health_check_timeout=600,
)

Once endpoint has been created, you would be able to perform real-time inference.

### B. Proofread and detect mistakes

**AI21 Grammatical Error Correction (GEC) model** offers access to our world-class proofreading engine. It has been specifically developed for providing suggestions on how to correct spelling mistakes, grammar, punctuation, word misuse, and more.

This model takes a piece of text and returns a list of suggested corrections for detected mistakes, along with the type of mistake.

The input text should contain **no more than 500 characters**. The output will include all detected mistakes and suggested corrections.

The following is a simple example of detecting different types of grammatical errors and suggesting corrections for them.

First, create an AI21 Sagemaker client

In [ ]:
from ai21 import AI21SageMakerClient
client = AI21SageMakerClient(endpoint_name=endpoint_name)

In [ ]:
text = "Your grammer is a reflection of you image. Good or bad: you have made an an impression. And like all impressions, you in total control."

response = client.gec.create(
    text=text,
)

print(response.corrections)

Let's print the suggested corrections in a more comfortable way:

In [ ]:
print("Original text (with errors):")
print(text)
print("============================")
print("Suggested corrections:")
print("\n".join([
    "- {correctionType} error detected at position ({startIndex},{endIndex}): {originalText} --> {suggestion}".format(
        correctionType=x.correction_type,
        startIndex=x.start_index,
        endIndex=x.end_index,
        originalText=x.original_text,
        suggestion=x.suggestion
    ) for x in response.corrections]))

### Interested in learning more?
Take a look at our [guide](https://docs.ai21.com/docs/gec-api) to understand all the capabilities of AI21 Grammatical Error model

## 3. Clean-up

### A. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(endpoint_name)
model.sagemaker_session.delete_endpoint_config(endpoint_name)

### B. Delete the model

In [ ]:
model.delete_model()